![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Crypto-Arbitrage with Bellman-Ford Algorithm written in Python
## Definitions
> Triangular arbitrage is the result of a discrepancy between three foreign currencies that occurs when the currency's exchange rates do not exactly match up. These opportunities are rare and traders who take advantage of them usually have advanced computer equipment and/or programs to automate the process.[^1]

###### Graph:
> A graph is a combinatorial object composed of a set of vertices V (also known as nodes) and a set of edges E. The edges correspond to pairs of vertices, which are generally distinct, and without a notion of order in the sense where (u,v) and (v,u) denote the same edge.
At times, we consider a variant, the directed graph, where the edges have an ori- entation. In this case, the edges are usually known as arcs. The arc (u,v) has origin u and destination v. Most of the algorithms described in this book operate on directed graphs but can be applied to non-directed graphs by replacing each edge (u,v) by two arcs (u,v) and (v,u).
Graphs can contain additional information, such as weights or letters, in the form of labels on the vertices or the edges.

###### Bellman-Ford algorithm:
> The Bellman-Ford algorithm finds the minimum weight path from a single source vertex to all other vertices on a weighted directed graph.

Our goal is to develop a systematic method for detecting arbitrage opportunities by framing the problem in the language of graphs. 

## Approach
We will assign currencies to different vertices, and let the edge weight represent the exchange rate.
Find a cycle in the graph such that multiplying all the edge weights along that cycle results in a value greater than 1. In fact we have already described an algorithm that can find such a path – the problem is equivalent to finding a negative-weight cycle, provided we do some preprocessing on the edges.

We note that Bellman-Ford computes the path weight by adding the individual edge weights. To make this work for exchange rates, which are multiplicative, a fix is to first take the logs of all the edge weights. Thus when we sum edge weights along a path we are actually multiplying exchange rates – we can recover the multiplied quantity by exponentiating the sum. Secondly, Bellman-Ford attempts to find minimum weight paths and negative edge cycles, whereas our arbitrage problem is about maximising the amoun t of currency received. Thus as a simple modification, we must also make our log weights negative.
Now we are able to apply Bellman-Ford. The minimal weight between two currency vertices corresponds to the optimal exchange rate, the value of which can be found by by exponentiating the negative sum of weights along the path. A negative-weight cycle on the negative-log graph corresponds to an arbitrage opportunity.

## Code
> List of functions:
```
get_price()
```
- get last price from *QuantConnect[^3] API* and put together into a dataframe by using pandas
```
Trading.strategy()
```
- recall the `Graph` class[^2] and the `Graph.bellman_ford()` to perform the strategy and print the  *boolean* variable `bol` (`True` if negative cycles were detected, `False` otherwise) and the **profit**  expressed as %



[^1]:[Investopedia: Triangular Arbitrage](https://www.investopedia.com/terms/t/triangulararbitrage.asp)\
[^2]: The code to implement it has been taken from this [book](https://amzn.to/3bBI8tP)\
[^3]: [Datasets from QuantConnect]https://www.quantconnect.com/docs/v2/research-environment/datasets/crypto


##### Importing modules	

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()


##### Defining function for getting data	

In [ ]:

    
def get_data(sym1:str,sym2:str, sym3:str , start_time: datetime, end_time: datetime):
    """
    @sym1: ticker name
    @sym2: ticker name
    @sym3: ticker name
    @start_time: date to start getting data
    @end_time: last day of getting data
    """

    ticker1 = qb.AddCrypto(sym1).Symbol
    ticker2 = qb.AddCrypto(sym2).Symbol
    ticker3 = qb.AddCrypto(sym3).Symbol

    price1 = qb.History(ticker1, start_time, end_time, Resolution.Tick)['lastprice']
    price2 = qb.History(ticker2, start_time, end_time, Resolution.Tick)['lastprice']
    price3 = qb.History(ticker3, start_time, end_time, Resolution.Tick)['lastprice']

    df = pd.DataFrame({'symbol':list(),'price':list()})
    price1 = -np.log(price1.values.tolist())
    price2 = -np.log(price2.values.tolist())
    price3 = -np.log(price3.values.tolist())

    df=df.append([{'symbol': sym1 + '_' + str(price1.index()[-1]) , 'price': price1[-1]}], ignore_index=True)
    df=df.append([{'symbol': sym2 + '_' + str(price2['Date'][-1]), 'price': price2[-1]}], ignore_index=True)
    df=df.append([{'symbol': sym3 + '_' + str(price3['Date'][-1]) , 'price': price3[-1]}], ignore_index=True)
    return df


##### Create the Graph class	

In [ ]:

class Graph:
    def __init__(self):
        self.neighbors = []
        self.name2node = {}
        self.node2name = []
        self.weight = []
    
    def __len__(self):
        return len(self.node2name)
    def __getitem__(self,v):
        return self.neighbors[v]
    
    def add_node(self,name):
        assert name not in self.name2node
        self.name2node[name] = len(self.name2node)
        self.node2name.append(name)
        self.neighbors.append([]) 
        self.weight.append({})
        return self.name2node[name]
    
    def add_edge(self,name_u,name_v,weight_uv=None):
        self.add_arc(name_u, name_v, weight_uv) 
        self.add_arc(name_v, name_u, weight_uv)

    def add_arc(self,name_u,name_v,weight_uv=None):
        u = self.name2node[name_u]
        v = self.name2node[name_v] 
        self.neighbors[u].append(v)
        self.weight[u][v] = weight_uv

    def bellman_ford(self, weight, source=0):
        graph = self
        n = len(graph)
        dist = [float('inf')] * n
        prec = [None]*n
        dist[source] = 0
        for nb_iterations in range(n):
            changed = False
            for node in range(n):
                for neighbor in graph[node]:
                    alt = dist[node] + weight[node][neighbor]
                    if alt < dist[neighbor]:
                        dist[neighbor] = alt
                        prec[neighbor] = node
                        changed = True
                if not changed:
                    return dist,prec,False
        return dist, prec, True




##### Defining the strategy	

In [ ]:
def strategy(df: pd.DataFrame):
        """
        df: DataFrame of prices
        """
        global g
        g = Graph()
        for i in df.symbol:
            g.add_node(i)
        for j in df.price:
            g.weight.append((j))
        
        for m in range(len(df)-1):
            g.add_arc(df.symbol[m],df.symbol[m+1],df.price[m])
        for n in reversed(range(len(df)-1)):
            g.add_arc(df.symbol[n],df.symbol[n+1],df.price[n])
        
        dist, prec, bol = g.bellman_ford(g.weight,source=0)
        #####
        tot = 0
        for i in dist:
            tot *= i
        profit = np.exp(-tot)-1
        if bol:
            print(f"Profit from the strategy is: {profit*100:.2g}%\n")
        return bol, profit



##### implementing the strategy	

In [ ]:
import time 
start_time = datetime.datetime(2023, 5, 1)
end_time = datetime.datetime(2023, 5, 3)
for i in range(100):
    df = get_data("BTCUSD","ETHUSD","LTCUSD",start_time,end_time)
    print(df)
    print(strategy(df))
    time.sleep(5)

##### Final Consideration:
This algorith was built for only three cryptocurrencies, but you can modify the code to apply with real time data and most important: **multi currencies**. You can actually add as many tickers as you want to test this algorithm (you need a proper subscription) and if you want to trade in a seconds environment, remember to change the code accordingly 	